In [1]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tuning_model_class import w2v_tuning
from transformers import  ElectraTokenizerFast
from nltk.corpus import wordnet, stopwords, words
import json, sqlparse, time
import pandas as pd
import numpy as np

[nltk_data] Downloading package words to
[nltk_data]     /Users/thanawatthongpia/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thanawatthongpia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/thanawatthongpia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


run


In [2]:
vocabulary = words.words()

w2vmodel = w2v_tuning()
w2vmodel.set_vector_path("w2v.42B.300d.txt",'GloVe.wordvectors')
w2vmodel.CreateW2V_Model()

load pre-train model


In [3]:
w2vmodel.model.wv.most_similar('event',topn=5)

[('events', 0.8563973903656006),
 ('venue', 0.6610988974571228),
 ('weekend', 0.6492050290107727),
 ('celebration', 0.6340488195419312),
 ('show', 0.620791494846344)]

In [4]:
pointx_cols = pd.read_csv('src/pointx_fbs_rpt_dly.csv').columns.to_list()
compare_df = pd.read_csv('src/compare_result.csv')
questions = compare_df['Question'].to_list()
sql_queries = compare_df['SQL'].to_list()

In [5]:
def preprocess_question(text:str):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [WordNetLemmatizer().lemmatize(word, wordnet.NOUN) for word in tokens if (word not in stop_words and word.isalnum())]
    return filtered_tokens

In [6]:
def tokenize_column(col_list:list):
    tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-large-discriminator")
    token_col = [tokenizer.tokenize(col_name) for col_name in col_list]
    return token_col

In [7]:
def get_col(sql):
    col = []
    ignore = ['avg_cnt desc','avg_cnt','over','extract','desc','datediff','dayofweek','cnt','dateadd','max','min','sum','count','getdate','timestampdiff','weekday','having','month','year','day','date','avg','team_tds.tds_intern.pointx_fbs_txn_rpt_dly']
    for token in sql.tokens:
        # print(type(token),token)
        if str(token).lower() in ignore:
            continue
        if isinstance(token, sqlparse.sql.Identifier):
            if len(str(token).lower().split('as')) > 1:
                col += get_col(token)
            elif '"' in str(token): #ignore condition value
                continue
            else:
                col.append(str(token))
        elif isinstance(token, sqlparse.sql.IdentifierList) or isinstance(token, sqlparse.sql.Where) or isinstance(token, sqlparse.sql.Having) or isinstance(token, sqlparse.sql.Comparison) or isinstance(token, sqlparse.sql.Function) or isinstance(token, sqlparse.sql.Parenthesis) or isinstance(token,sqlparse.sql.Operation):
            col += get_col(token)
        if str(token).lower() not in pointx_cols:
            continue
            
    return col

In [8]:
# tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-large-discriminator")

def compare_word_col(df, question:str):
    # print(question)
    q_tokens = preprocess_question(question)
    for qtoken in q_tokens:
        is_value_exist = df['Q_token'].isin([q_tokens])
        if not is_value_exist.any():    #word is not exist
            list_score = [qtoken]
            for col_name in list(df.columns):
                if col_name == "Q_token":
                    continue
                score = 0
                for ctoken in col_name.split('_'):#tokenizer.tokenize(col_name):
                    # print(ctoken)
                    try:
                        similar_score = w2vmodel.model.wv.similarity(ctoken,qtoken)
                    except KeyError:
                        similar_score = 0
                    if score < similar_score:
                        score = similar_score
                
                list_score.append(score)
                
            df.loc[len(df.index)] = list_score
            # Reset the index to reindex the DataFrame
            df.reset_index(drop=True, inplace=True)


    return df

In [9]:
cols = ['Q_token'] + pointx_cols
df = pd.DataFrame(columns=cols)
score_df = compare_word_col(df,questions[0])
score_df

,Q_token,event_date,event_month,event_bundle_sequence_id,event_timestamp,event_name,customer_id,user_pseudo_id,user_id,event_previous_timestamp,...,total_points,transaction_id,transaction_status,transaction_type,unable_to_proceed,update_with_analytics,client_code,client_member_id,_dl_load_ts,_date
0,many,0.496840,0.516270,0.496840,0.496840,0.516184,0.466726,0.425577,0.425577,0.584997,...,0.484565,0.353882,0.452535,0.518135,0.634550,0.651627,0.442809,0.442809,0.410492,0.398094
1,daily,0.371336,0.543651,0.371336,0.371336,0.371336,0.383368,0.359119,0.359119,0.455470,...,0.435335,0.252077,0.346206,0.331633,0.471493,0.468382,0.362503,0.362503,0.341183,0.323847
2,active,0.413306,0.440023,0.399785,0.383472,0.398787,0.399785,0.493628,0.493628,0.440574,...,0.482014,0.399785,0.496258,0.482028,0.490440,0.492921,0.409255,0.541228,0.366661,0.413306
3,user,0.432669,0.419145,0.501427,0.419145,0.518544,0.603627,1.000000,1.000000,0.540777,...,0.466737,0.501427,0.493277,0.493277,0.469058,0.498254,0.607165,0.607165,0.482067,0.432669
4,day,0.614402,0.739821,0.614402,0.614402,0.614402,0.431944,0.363407,0.363407,0.614402,...,0.463473,0.340255,0.393026,0.460299,0.589244,0.567759,0.385363,0.398327,0.411428,0.524131


In [26]:
result_df = pd.DataFrame(columns=['Question',
                                  'SQL',
                                  'Expect Columns',
                                  'Columns-token Min threshold',
                                  'Columns-token choose columns'])

In [27]:
n_cols = 0
start_time = time.time()
for i, q in enumerate(questions):
    print("QUESTION:\t",q)
    ex_match = {}
    pt_match = {}
    q_tokens = preprocess_question(q)
    cols = ['Q_token'] + pointx_cols
    df = pd.DataFrame(columns=cols)
    score_df = compare_word_col(df,q)
    numeric_score_df = score_df.iloc[:, 1:]

    sql_parse = sqlparse.parse(sql_queries[i])[0]
    col_labels = list(set(get_col(sql_parse)))
    
    print("COLUMNS LABEL:\t",col_labels)
    try:
        min_threshold = numeric_score_df[col_labels].min().min()
    except KeyError:        #no column label (human error)
        col_labels = [c for c in col_labels if c in pointx_cols]
        min_threshold = numeric_score_df[col_labels].min().min()
    print("MIN THRESHOLD:\t",min_threshold)
    n_choose_column = len(numeric_score_df.min()[numeric_score_df.min() > min_threshold])
    print("NUMBER OF COLUMNS:\t",n_choose_column)
    n_cols += n_choose_column
    print()
    result_df.loc[len(result_df.index)] = [q,
                                           sql_queries[i],
                                           col_labels,
                                           min_threshold,
                                           n_choose_column]

print("AVERAGE NUMBER OF COLUMNS:\t",n_cols/len(questions))
print("AVG TIME PER QUESTION:\t",(time.time()-start_time)/len(questions))

QUESTION:	 How many daily active users each day?
COLUMNS LABEL:	 ['event_date', 'ga_session_id', 'user_pseudo_id', 'engagement_time_msec']
MIN THRESHOLD:	 0.3538821
NUMBER OF COLUMNS:	 125

QUESTION:	 How many monthly active users each month?
COLUMNS LABEL:	 ['ga_session_id', 'event_month', 'user_pseudo_id', 'engagement_time_msec']
MIN THRESHOLD:	 0.3538821
NUMBER OF COLUMNS:	 108

QUESTION:	 What is the average number of daily active users last 7 days?
COLUMNS LABEL:	 ['event_date', 'ga_session_id', 'user_pseudo_id', 'engagement_time_msec']
MIN THRESHOLD:	 0.29360616
NUMBER OF COLUMNS:	 146

QUESTION:	 What is the mean number of daily active users from last 30 days?
COLUMNS LABEL:	 ['event_date', 'ga_session_id', 'user_pseudo_id', 'engagement_time_msec']
MIN THRESHOLD:	 0.3515815
NUMBER OF COLUMNS:	 125

QUESTION:	 When was the last time each user was active?
COLUMNS LABEL:	 ['user_pseudo_id', 'ga_session_id', 'event_timestamp', 'event_date', 'engagement_time_msec']
MIN THRESHOLD:	 0.

In [11]:
#result.csv

In [12]:
from sentence_transformers import SentenceTransformer, util

description_df = pd.read_excel('src/New_query_Description.xlsx',header=1)
description_df = description_df[['Column','Description']].dropna().reset_index(drop=True)
descriptions = description_df['Description'].tolist()

In [41]:
model = SentenceTransformer('all-MiniLM-L6-v2')
description_embs = [model.encode(des) for des in descriptions]
descp_threshold = []
descp_n_cols = []
def get_col_max_score(question,col_labels,description_embs=description_embs,description_df=description_df):
    q_emb = model.encode(question)
    scores = np.array([float(util.cos_sim(q_emb, des)) for des in description_embs])
    col_labels_index = description_df[description_df['Column'].isin(col_labels)].index.tolist()
    col_labels_score = scores[col_labels_index]
    min_threshold = np.min(col_labels_score)

    print("QUESTION:\t",question)
    print("EXPECT COLUMNS:\t",col_labels)
    print("MIN THRESHOLD:\t",min_threshold)
    # print("MAX THRESHOLD:\t",np.max(col_labels_score))
    # print("MAX SCORE COLUMN:\t",description_df.iloc[np.argmax(scores)]['Column'])
    # print("DESCIPTION:\t",description_df.iloc[np.argmax(scores)]['Description'])
    # print("SCORE:\t",np.max(scores))
    n_columns = len(description_df.iloc[np.where(scores >= min_threshold)])
    print(f"CHOOSE RELATE COLUMN WITHIN THRESHOLD (FROM {len(scores)} COLUMNS):",n_columns)
    print()
    descp_threshold.append(min_threshold)
    descp_n_cols.append(n_columns)
    
    return n_columns

In [42]:
n_cols = 0
start_time = time.time()
for i, q in enumerate(questions):
    sql_parse = sqlparse.parse(sql_queries[i])[0]
    col_labels = list(set(get_col(sql_parse)))
    n_cols += get_col_max_score(q,col_labels)

print("AVERAGE NUMBER OF COLUMNS:\t",n_cols/len(questions))
print("AVG TIME PER QUESTION:\t",(time.time()-start_time)/len(questions))

QUESTION:	 How many daily active users each day?
EXPECT COLUMNS:	 ['event_date', 'ga_session_id', 'user_pseudo_id', 'engagement_time_msec']
MIN THRESHOLD:	 0.15984684228897095
CHOOSE RELATE COLUMN WITHIN THRESHOLD (FROM 182 COLUMNS): 49

QUESTION:	 How many monthly active users each month?
EXPECT COLUMNS:	 ['ga_session_id', 'event_month', 'user_pseudo_id', 'engagement_time_msec']
MIN THRESHOLD:	 0.1179906576871872
CHOOSE RELATE COLUMN WITHIN THRESHOLD (FROM 182 COLUMNS): 68

QUESTION:	 What is the average number of daily active users last 7 days?
EXPECT COLUMNS:	 ['event_date', 'ga_session_id', 'user_pseudo_id', 'engagement_time_msec']
MIN THRESHOLD:	 0.11249055713415146
CHOOSE RELATE COLUMN WITHIN THRESHOLD (FROM 182 COLUMNS): 76

QUESTION:	 What is the mean number of daily active users from last 30 days?
EXPECT COLUMNS:	 ['event_date', 'ga_session_id', 'user_pseudo_id', 'engagement_time_msec']
MIN THRESHOLD:	 0.15479889512062073
CHOOSE RELATE COLUMN WITHIN THRESHOLD (FROM 182 COLUMNS

In [43]:
result_df['Description Min threshold'] = descp_threshold
result_df['Description choose columns'] = descp_n_cols

In [44]:
result_df

,Question,SQL,Expect Columns,Columns-token Min threshold,Columns-token choose columns,Description Min threshold,Description choose columns
0,How many daily active users each day?,"SELECT event_date, COUNT(DISTINCT user_pseudo_...","[event_date, ga_session_id, user_pseudo_id, en...",0.353882,125,0.159847,49
1,How many monthly active users each month?,"SELECT event_month, COUNT(DISTINCT user_pseudo...","[ga_session_id, event_month, user_pseudo_id, e...",0.353882,108,0.117991,68
2,What is the average number of daily active use...,"SELECT AVG(cnt) FROM\n (SELECT event_date, CO...","[event_date, ga_session_id, user_pseudo_id, en...",0.293606,146,0.112491,76
3,What is the mean number of daily active users ...,"SELECT AVG(cnt) FROM\n (SELECT event_date, CO...","[event_date, ga_session_id, user_pseudo_id, en...",0.351582,125,0.154799,52
4,When was the last time each user was active?,"SELECT user_pseudo_id, MAX(last_active) FROM\n...","[user_pseudo_id, ga_session_id, event_timestam...",0.383472,139,0.258170,31
...,...,...,...,...,...,...,...
62,What's the average sessions duration for users...,"SELECT event_date, AVG(time_difference_second...","[user_pseudo_id, event_timestamp, event_name, ...",0.301989,115,0.075776,99
63,What's the percentage of users who have used t...,SELECT\n (COUNT(DISTINCT CASE WHEN WEEKDAY(ev...,[user_pseudo_id],0.259780,146,0.230391,24
64,How many users have performed a specific event...,SELECT COUNT(DISTINCT user_pseudo_id)\nFROM te...,"[event_date, user_pseudo_id]",0.336081,99,0.177395,43
65,How many users have opened the app on at least...,SELECT SUM(user_with_multiple_devices) AS tota...,"[device_category, user_pseudo_id]",0.312540,139,0.396473,8


In [40]:
result_df.to_csv('smilarity_result.csv',index=False)